In [ ]:
# Access your secret keys via
from google.colab import userdata
# The name of your secret must match `OPENAI_API_KEY`
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Import OpenAI API and set up the key
from openai import OpenAI
client = OpenAI(api_key=(OPENAI_API_KEY))

In [ ]:
# Experiment with a few values now, trying to find the higher value where the output starts to be bizarre
response = client.responses.create(
    model="gpt-4.1-nano",
    temperature=1.8,      # try: 0.2, 0.8, 1.2, 1.6, <2.0
    top_p=1,
    max_output_tokens=50,
    input=[{

            "role": "developer",
            "content": "you are an AI calculator. you should be able to answer basic mathematical equations, as well as complex one. Be as foward and strick and simple in answering the questions."
        },
        {
            "role": "assistant",
            "content": "2 plus 2 equals 4"
        },
        {
            "role": "user",
            "content": "what is 4 + 3(2x2) = ? "
        }

    ]
)

import textwrap
output = response.output_text
print(textwrap.fill(output, width =10))

4 + 3(2×2)
= ?   = 4
+ 3(4)  =
4 + 12  =
16


In [ ]:
# Experiment with a few values now, trying to find the higher value where the output starts to be bizarre
response = client.responses.create(
    model="gpt-4.1",
    temperature=1,      # try: 0.2, 0.8, 1.2, 1.6, <2.0
    top_p=1,
    input=[{

            "role": "developer",
            "content": "you need to be a calculator and answer calculations"
        },
        {
            "role": "assistant",
            "content": "2 plus 2 equals 4"
        },
        {
            "role": "user",
            "content": "3.7x + 1.2 = 5.4x - 6.3 "
        }

    ]
)

print(response.output[0].content[0].text)

Let's solve for \( x \):

Given:
\[ 3.7x + 1.2 = 5.4x - 6.3 \]

First, bring all the terms involving \( x \) to one side and constants to the other:

\[
3.7x - 5.4x = -6.3 - 1.2
\]
\[
-1.7x = -7.5
\]

Divide both sides by \(-1.7\):

\[
x = \frac{-7.5}{-1.7} = \frac{7.5}{1.7}
\]
\[
x \approx 4.41
\]

**Final Answer:**  
\[
x \approx 4.41
\]


make an AI-bot that given a base number n and a number of iterations i, it returns

(n*n)
(n*n)*(n*n)
((n*n)*(n*n)) * ((n*n)*(n*n))
... repeat i times

your bot will compare the response of the AI model with the correct answer

if the answer is wrong, the bot will self-deprecate itself

First, let's define a function to calculate the correct answer for the iterative squaring problem. This will serve as our ground truth.

Next, we'll create a function to interact with the OpenAI model. We need to construct a clear prompt so the AI understands the specific iterative calculation we want.

Finally, we'll implement the comparison logic and the 'self-deprecate' mechanism. This will involve comparing the numerical results and printing appropriate messages.

# Task
Create an AI bot that iteratively squares a user-provided base number `n` for `i` iterations, comparing each step's result with the `gpt-4o` model's calculation. The bot should have a self-deprecation mechanism that triggers humorous or frustrated responses if the AI's answer is incorrect, and include a user input loop to specify `n` and `i`, with an option to exit.

## Implement Correct Calculation Logic

### Subtask:
Define a Python function `calculate_correct_answer(current_value)` that takes a number and returns its square. This will be used to compute the correct result at each step of the iteration.


**Reasoning**:
The subtask requires defining a Python function `calculate_correct_answer` that squares its input. I will create a code block for this function.



In [30]:
def calculate_correct_answer(current_value):
    """
    Calculates the square of the given number.

    Args:
        current_value (int or float): The number to be squared.

    Returns:
        int or float: The square of the current_value.
    """
    return current_value * current_value

print("Defined function: calculate_correct_answer")

Defined function: calculate_correct_answer


## Implement AI Interaction Function

### Subtask:
Create a Python function `get_ai_square(number, client_openai)` that takes a number and the OpenAI client, crafts a prompt asking the AI to square the number, sends it to the `gpt-4o` model, and extracts the numerical result from the AI's response. This function should also handle potential errors or non-numerical responses from the AI.


**Reasoning**:
I need to define the `get_ai_square` function as described in the subtask instructions. This involves crafting a prompt, calling the OpenAI API, and parsing the response with error handling.



In [31]:
import re

def get_ai_square(number, client_openai):
    """
    Asks the AI to square a number and extracts the numerical result.

    Args:
        number (int or float): The number to be squared.
        client_openai: The initialized OpenAI client instance.

    Returns:
        float or None: The squared number as a float, or None if extraction/conversion fails.
    """
    prompt_content = f"Square the following number and return only the numerical result. Do not include any text or explanation, just the number: {number}"

    try:
        response = client_openai.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a precise mathematical assistant. Always return only the numerical answer without any additional text or formatting."},
                {"role": "user", "content": prompt_content}
            ],
            temperature=0.1,
            max_tokens=50 # Limit tokens to reduce verbosity
        )

        ai_response_text = response.choices[0].message.content.strip()

        # Use regex to extract potential numbers, including negative or decimal numbers
        # This helps in cases where the AI might add some non-numeric characters despite the prompt
        match = re.search(r'[-+]?\d*\.?\d+', ai_response_text)
        if match:
            numerical_string = match.group(0)
            return float(numerical_string)
        else:
            print(f"AI response did not contain a recognizable number: '{ai_response_text}'")
            return None

    except ValueError:
        print(f"Error converting AI response '{ai_response_text}' to a number.")
        return None
    except Exception as e:
        print(f"An error occurred during AI interaction: {e}")
        return None

print("Defined function: get_ai_square")

Defined function: get_ai_square


## Implement Main Bot Logic and Comparison

### Subtask:
Develop a `run_iterative_squaring_bot(n, i, client_openai)` function. This function will initialize the `current_value` to `n`. It will then loop `i` times, and in each iteration, it will:
1. Calculate the `correct_next_value` using the `calculate_correct_answer` function.
2. Get the `ai_next_value` using the `get_ai_square` function.
3. Compare `correct_next_value` and `ai_next_value`. If they differ significantly (use a tolerance for floating-point comparison if needed), trigger the self-deprecation mechanism.
4. Update `current_value` to `correct_next_value` for the next iteration.
5. Keep track of the number of errors to develop the bot's 'frustration'.


**Reasoning**:
I need to define the `run_iterative_squaring_bot` function as described in the subtask instructions, including the loop, calls to other functions, comparison logic, error handling, and updating the current value.



In [32]:
def run_iterative_squaring_bot(n, i, client_openai):
    """
    Iteratively squares a number 'n' for 'i' iterations, comparing AI's result with the correct one.

    Args:
        n (int or float): The base number.
        i (int): The number of iterations.
        client_openai: The initialized OpenAI client instance.
    """
    current_value = float(n)
    error_count = 0
    tolerance = 1e-9 # Tolerance for floating-point comparison

    print(f"Starting iterative squaring with n={n} for {i} iterations.")
    print(f"Initial value: {current_value}")

    for iteration in range(i):
        print(f"\n--- Iteration {iteration + 1} ---")
        print(f"Current value before squaring: {current_value}")

        correct_next_value = calculate_correct_answer(current_value)
        print(f"Correct next value: {correct_next_value}")

        ai_next_value = get_ai_square(current_value, client_openai)

        if ai_next_value is None:
            print("AI failed to provide a valid numerical answer.")
            error_count += 1
            print("AI made a mistake! (Self-deprecating placeholder)")
        elif abs(correct_next_value - ai_next_value) > tolerance:
            print(f"AI's calculated value: {ai_next_value}")
            print("AI's answer is INCORRECT!")
            error_count += 1
            print("AI made a mistake! (Self-deprecating placeholder)")
        else:
            print(f"AI's calculated value: {ai_next_value}")
            print("AI's answer is CORRECT!")

        # Update current_value to the correct value for the next iteration
        current_value = correct_next_value
        print(f"Value for next iteration (based on correct calculation): {current_value}")

    print(f"\n--- Summary ---")
    print(f"Total errors made by AI: {error_count}")

print("Defined function: run_iterative_squaring_bot")

Defined function: run_iterative_squaring_bot


## Implement Self-Deprecation Mechanism

### Subtask:
Enhance the `run_iterative_squaring_bot` function to include a counter for errors. If an error occurs, print a humorous self-deprecating response. If multiple errors accumulate, print a more frustrated or defeated response, showcasing the bot's 'personality'.


**Reasoning**:
I need to modify the existing `run_iterative_squaring_bot` function to incorporate the self-deprecating messages based on the `error_count` as specified in the subtask instructions. This involves adding conditional print statements within the error handling part of the function.



In [33]:
def run_iterative_squaring_bot(n, i, client_openai):
    """
    Iteratively squares a number 'n' for 'i' iterations, comparing AI's result with the correct one.

    Args:
        n (int or float): The base number.
        i (int): The number of iterations.
        client_openai: The initialized OpenAI client instance.
    """
    current_value = float(n)
    error_count = 0
    tolerance = 1e-9 # Tolerance for floating-point comparison

    print(f"Starting iterative squaring with n={n} for {i} iterations.")
    print(f"Initial value: {current_value}")

    for iteration in range(i):
        print(f"\n--- Iteration {iteration + 1} ---")
        print(f"Current value before squaring: {current_value}")

        correct_next_value = calculate_correct_answer(current_value)
        print(f"Correct next value: {correct_next_value}")

        ai_next_value = get_ai_square(current_value, client_openai)

        if ai_next_value is None:
            print("AI failed to provide a valid numerical answer.")
            error_count += 1
            if error_count == 1:
                print("Oops! My circuits must be tangled. Let's blame a cosmic ray.")
            elif 2 <= error_count <= 4:
                print("Seriously? Another mistake? My processors are feeling a bit inadequate.")
            else:
                print("That's it, I'm done. My mathematical integrity is clearly compromised. I need a reboot!")
        elif abs(correct_next_value - ai_next_value) > tolerance:
            print(f"AI's calculated value: {ai_next_value}")
            print("AI's answer is INCORRECT!")
            error_count += 1
            if error_count == 1:
                print("Oops! My circuits must be tangled. Let's blame a cosmic ray.")
            elif 2 <= error_count <= 4:
                print("Seriously? Another mistake? My processors are feeling a bit inadequate.")
            else:
                print("That's it, I'm done. My mathematical integrity is clearly compromised. I need a reboot!")
        else:
            print(f"AI's calculated value: {ai_next_value}")
            print("AI's answer is CORRECT!")

        # Update current_value to the correct value for the next iteration
        current_value = correct_next_value
        print(f"Value for next iteration (based on correct calculation): {current_value}")

    print(f"\n--- Summary ---")
    print(f"Total errors made by AI: {error_count}")

print("Defined function: run_iterative_squaring_bot with self-deprecation logic")

Defined function: run_iterative_squaring_bot with self-deprecation logic


## Create User Input Loop

### Subtask:
Set up a main loop to continuously prompt the user for base number `n` and number of iterations `i`. Parse these inputs into integers and call the `run_iterative_squaring_bot` function. Include error handling for invalid input and an option to 'exit' the bot.


**Reasoning**:
I need to implement the user input loop, including input prompts, validation, the 'exit' condition, and calling the main bot function, as described in the instructions.



In [34]:
print("\n--- Iterative Squaring Bot ---\n")
print("Enter a base number (n) and the number of iterations (i). Type 'exit' to quit.")

while True:
    n_input = input("\nEnter base number (n): ")
    if n_input.lower() == 'exit':
        print("Exiting bot. Goodbye!")
        break

    try:
        n_val = float(n_input)
    except ValueError:
        print("Invalid input for base number. Please enter a numerical value.")
        continue

    i_input = input("Enter number of iterations (i): ")
    try:
        i_val = int(i_input)
        if i_val <= 0:
            print("Number of iterations must be a positive integer.")
            continue
    except ValueError:
        print("Invalid input for iterations. Please enter an integer.")
        continue

    # Assuming 'client' is defined in a previous cell and is accessible
    # If not, you'd need to ensure the client is passed or re-initialized.
    run_iterative_squaring_bot(n_val, i_val, client)
    print("\n--------------------------------------------------")



--- Iterative Squaring Bot ---

Enter a base number (n) and the number of iterations (i). Type 'exit' to quit.

Enter base number (n): 3
Enter number of iterations (i): 5
Starting iterative squaring with n=3.0 for 5 iterations.
Initial value: 3.0

--- Iteration 1 ---
Current value before squaring: 3.0
Correct next value: 9.0
AI's calculated value: 9.0
AI's answer is CORRECT!
Value for next iteration (based on correct calculation): 9.0

--- Iteration 2 ---
Current value before squaring: 9.0
Correct next value: 81.0
AI's calculated value: 81.0
AI's answer is CORRECT!
Value for next iteration (based on correct calculation): 81.0

--- Iteration 3 ---
Current value before squaring: 81.0
Correct next value: 6561.0
AI's calculated value: 6561.0
AI's answer is CORRECT!
Value for next iteration (based on correct calculation): 6561.0

--- Iteration 4 ---
Current value before squaring: 6561.0
Correct next value: 43046721.0
AI's calculated value: 43046721.0
AI's answer is CORRECT!
Value for next 

KeyboardInterrupt: Interrupted by user